In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, Point
import itertools

# --- GİRİŞ ---
image_path = r"image.png--path"

orijinal_resim = cv2.imread(image_path)
if orijinal_resim is None:
    print(f"HATA: '{image_path}' bulunamadı.")
    exit()

img_rgb = cv2.cvtColor(orijinal_resim, cv2.COLOR_BGR2RGB)
img_hsv = cv2.cvtColor(orijinal_resim, cv2.COLOR_BGR2HSV)

# --- ADIM 1: Kırmızı Yasaklı Bölgeleri Bul (Maske oluştur, ama resmi silme!) ---
lower_red1 = np.array([0, 100, 100])
upper_red1 = np.array([10, 255, 255])
mask1 = cv2.inRange(img_hsv, lower_red1, upper_red1)

lower_red2 = np.array([170, 100, 100])
upper_red2 = np.array([180, 255, 255])
mask2 = cv2.inRange(img_hsv, lower_red2, upper_red2)

kirmizi_maske = mask1 + mask2
kernel_genisletme = np.ones((15, 15), np.uint8)
kirmizi_maske_genis = cv2.dilate(kirmizi_maske, kernel_genisletme, iterations=1)


# --- ADIM 2: BÜTÜN Şekli Algılama (Sorunu çözdüğümüz yer) ---
gray = cv2.cvtColor(orijinal_resim, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (7, 7), 0)

# Hem siyah hem de kırmızı kalem beyaz kağıttan koyu olduğu için ikisini birden yakalayacağız
_, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Kalem izlerindeki ufak kopuklukları "kaynatmak" (birleştirmek) için morfolojik kapama
kernel_kapama = np.ones((9, 9), np.uint8)
thresh_closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel_kapama)

# Konturu şimdi bu "kesintisiz" halden buluyoruz
contours, _ = cv2.findContours(thresh_closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

en_buyuk_kontur = max(contours, key=cv2.contourArea)
poly_koordinatlari = en_buyuk_kontur.reshape(-1, 2)

gercek_parca = Polygon(poly_koordinatlari)
agirlik_merkezi = gercek_parca.centroid # Artık doğru yerde olacak!


# --- ADIM 3: Aday Noktaları Oluşturma ve Filtreleme ---
ornek_sayisi = 120 
ham_adaylar = [gercek_parca.exterior.interpolate(i/ornek_sayisi, normalized=True) for i in range(ornek_sayisi)]

temiz_adaylar = []
for p in ham_adaylar:
    px, py = int(p.x), int(p.y)
    if 0 <= py < kirmizi_maske_genis.shape[0] and 0 <= px < kirmizi_maske_genis.shape[1]:
        if kirmizi_maske_genis[py, px] == 0:
            temiz_adaylar.append(p)
    else:
        temiz_adaylar.append(p)


# --- ADIM 4: Optimizasyon ---
en_iyi_klempler = None
en_yuksek_skor = 0

for komb in itertools.combinations(temiz_adaylar, 3):
    destek_ucgeni = Polygon(komb)
    skor = destek_ucgeni.area
    if destek_ucgeni.contains(agirlik_merkezi):
        skor *= 2.5
    if skor > en_yuksek_skor:
        en_yuksek_skor = skor
        en_iyi_klempler = komb


# --- ADIM 5: Görselleştirme ---
sonuc_resmi = img_rgb.copy()

overlay = sonuc_resmi.copy()
overlay[kirmizi_maske_genis > 0] = (255, 0, 255) 
cv2.addWeighted(overlay, 0.4, sonuc_resmi, 0.6, 0, sonuc_resmi)

cv2.drawContours(sonuc_resmi, [en_buyuk_kontur], -1, (0, 255, 0), 3)
cv2.circle(sonuc_resmi, (int(agirlik_merkezi.x), int(agirlik_merkezi.y)), 10, (255, 0, 0), -1)

if en_iyi_klempler:
    kx, ky = [], []
    for p in en_iyi_klempler:
        px, py = int(p.x), int(p.y)
        kx.append(px)
        ky.append(py)
        cv2.drawMarker(sonuc_resmi, (px, py), (0, 0, 255), markerType=cv2.MARKER_SQUARE, markerSize=18, thickness=4)
    
    ucgen_pts = np.array([list(zip(kx, ky))], np.int32)
    cv2.polylines(sonuc_resmi, [ucgen_pts], isClosed=True, color=(0, 100, 255), thickness=2, lineType=cv2.LINE_AA)

plt.figure(figsize=(12, 8))
plt.title("Düzeltilmiş Kontur Tespiti ve Optimum Klempleme")
plt.imshow(sonuc_resmi)
plt.axis('off')
plt.show()